# Classes

- Simple class example...

In [ ]:
class Entity:
    """A simple class"""
    
    highest_risk_score = 0 # Class variable shared between all class instances
    
    def __init__(self, name, risk_score):
        """This gets called to create a new class"""
        local_variable = "hello" # Local variable with scope limited to __init__
        self.name = name 
        self.risk_score = risk_score
        Entity.highest_risk_score = max(risk_score, Entity.highest_risk_score)
        
    def example_function(self):
        print(f"{self.name} has a risk score of: {self.risk_score}")

### A Note about self...
- Self is used to describe variables specific to a singular class instance   
- so the self.name is specific to the created class instance      
- This is why sally and john can have different names stored as shown   
- Self is not a reserve keyword, you can use whatever name you like as the first variable    
- ... but self is used because it's general practice


### Self is what's passed automatically when calling a function from a class instance... 
- sally.example_function() == Person.example_function(sally) 


In [ ]:
john = Entity('John', 10)
sally = Entity('Sally', 99)

- Class instances are stored in a dictionary format
- vars(instance) works too

In [ ]:
john.__dict__

In [ ]:
Entity.__dict__

In [ ]:
john.name = "Johnny"
john.name

In [ ]:
sally.example_function()

- Class variable can be reached without instances

In [ ]:
print(Entity.highest_risk_score)

- Update class variable for all class instances


In [ ]:
Entity.highest_risk_score = "Shared Between all class instances"

In [ ]:
print(sally.highest_risk_score)
print(john.highest_risk_score)

- Can create attributes on the fly without impacting other instances...
- Can potentially be a very dangerous operation with typos leading to downstream issues

In [ ]:
sally.names = "Sal"
vars(sally)

In [ ]:
vars(john)

# Magic Methods
- We can use special operations and names spaces to improve flow and readability
- Unlocks a lot of power by being able to use builtin terminology instead haphazard namespaces
- https://docs.python.org/3/reference/datamodel.html


### Str, repr, format
- \__str__ is called when you print(an_object)
- \__repr__ is the interactive / representation view, but will also print if \__str__ is not defined
- \__format__ output... If not declared \__str__ will be the default returned value

In [ ]:
class Entity():
    
    def __init__(self, name, risk_score):
        self.name = name 
        self.risk_score = risk_score
    
    def __repr__(self):
        """Just as an example..."""
        return f"<Entity object: {self.name}>"
    
    def __str__(self):
        """Return a string of all items in instance dictionary"""
        return self.name

    def __format__(self, fmt):
        """Logic for f strings and .format"""
        if fmt == "%d":
            return f"{self.risk_score}"
        else:
            return f"{self.name}: {self.risk_score}"

In [ ]:
sample = Entity('John', 10)
print(sample)

- Defining repr method will improve object displays

In [ ]:
list_ = [sample, 1, "test"]
list_

In [ ]:
sample

In [ ]:
sample = Entity('John', 10)
print(sample)

- Adding in some custom logic to show format...

In [ ]:
print(f"Risk Score {sample:%d}") # Same as "{:%d}".format(sample)
print(f"Risk Score {sample}")

### Bools
- if \__bool__() is defined, use logic
- else if \__len__() is defined return False if len is zero
- Otherwise returns True

In [ ]:
class Entity():

    threshold = 75
    
    def __init__(self, name, risk_score):
        self.name = name 
        self.risk_score = risk_score
    
    def __bool__(self):
        """Return true if risky entity, false if not"""
        if self.risk_score > self.threshold:
            return True
        else:
            return False

- Can take advantage of using builtins to make code more readable
- Name variables and functions so you code is like reading text 

In [ ]:
def mitigate(entity):
    """Updates database to limit user activity"""
    print(f"Logic now running to mitigation against {entity.name}")

In [ ]:
employees = [Entity('John', risk_score=90), Entity('Sally', risk_score=10)]

for potentially_risky_entity in employees:
    if potentially_risky_entity:
        mitigate(potentially_risky_entity)

- Reads significantly better than something along these lines, which really isn't awful

### Len
- Override the meaning of len(instance)
- Keeps us from having a variety of various functions to guess about.. instance.number_of_events isn't pretty
- While below is just an example, make len as intuitive as possible for your classes

In [ ]:
class Entity():
    
    def __init__(self, name, risk_score, events):
        self.name = name 
        self.risk_score = risk_score
        self.associated_events = events

    def __len__(self):
        """
        What is returned when len() is called
        In this case return the length of associated events...
        """
        return len(self.associated_events)

In [ ]:
sample = Entity('John', 10, ["Network Event", "Physical Access", "Security Event"])

In [ ]:
len(sample)

### Iteration
- Defining \__getitem__ or \__iter__ allow you to make for loops over classes
- \__next__ can also be used with \__iter__ returning self for the same impact, which also allows you to call next on instances
- \__getitem__ has a corresponding \__missing__ method for keyed items 


In [ ]:
class Entity():
    
    def __init__(self, name, risk_score, events):
        self.name = name 
        self.risk_score = risk_score
        self.associated_events = events
    
    def __getitem__(self, index):
        """Naturally there would be more handling here such as index errors etc..."""
        return self.associated_events[index]    
    

In [ ]:
sample = Entity('John', 11, ["Network Event", "Physical Access", "Security Event"])
sample[2]

In [ ]:
for item in sample:
    print(item)

In [ ]:
class Entity():
    
    def __init__(self, name, risk_score, events):
        self.name = name 
        self.risk_score = risk_score
        self.associated_events = events
    
    #def __getitem__(self, index):
    #    """Naturally there would be more handling here such as index errors etc..."""
    #    print("inside getitem")
    #    return self.associated_events[index]   
    
    def __iter__(self):
        print("This is from __iter__:")
        yield from self.associated_events
                    

In [ ]:
sample = Entity('John', 11, ["Network Event", "Physical Access", "Security Event"])

for item in sample:
    print(item)

### Membership checks
- Having getitem or iter allow for membership checks, but \__contains__ can be used to fine tune this logic
- Contains will also likely be fasted depending on the logic... O(N) vs a hash map
- We can allow iter and contains to point at different data as well as seen below...

In [ ]:
class Entity():
    
    def __init__(self, name, aliases, risk_score, events):
        self.name = name 
        self.aliases = aliases
        self.risk_score = risk_score
        self.associated_events = events
        
    #def __contains__(self, value):
    #    """Overrides the 'in' check for objects"""
    #    return bool(value in self.aliases)  
    
    def __iter__(self):
        print("This is from __iter__:")
        yield from self.associated_events
                    

In [ ]:
sample_entity = Entity(name = 'John', 
                       aliases = set(["Johnny", "J"]), 
                       risk_score = 11, 
                       events = ["Network Event", "Physical Access", "Security Event"])

In [ ]:
question = "Network Event"
if question in sample_entity:
    print("Do something here..")

### Speciality comparisons 

- =>, >, ==, !=, etc are all easily set with corresponding functions
- We can use functools.total_ordering decorator to limit writing all of these 
- Just define \__eq__ and \__lt__ and you have the rest accounted for 

In [ ]:
from functools import total_ordering

@total_ordering
class Entity():
    
    def __init__(self, name, risk_score):
        self.name = name 
        self.risk_score = risk_score
        
    def __lt__(self, other):
        """Implement < operator"""
        return self.risk_score < other.risk_score

    def __eq__(self, other):
        """Implement == operator"""
        if type(other) is type(self):
            return self.risk_score == other.risk_score
        else:
            return False

In [ ]:
john = Entity('John', 50)
sally = Entity('Sally', 49)

if john > sally:
    print("yes")

### Special methods for handling getting and setting attribute values
- getattr called when an attribute does not exist 
- setattr called anytime an attribute is attempted to be set
- getattribute called every time an attribute is requested
- Generally you will not call getattribute, also very easy to cause infinite recursion by accident

In [ ]:
class Entity():

    valid_keys = ("name", "risk_score")
    
    def __init__(self, name, risk_score):
        self.name = name 
        self.risk_score = risk_score
    
    def __getattr__(self, name):
        print(f"{name} is not the attribute you are looking for...")
        return "I could return this however"
        
    #def __getattribute__(self, name):
    #    print(f"__getattribute__ called for {name}....")
    #    return super(Entity, self).__getattribute__(name)
        
    def __setattr__(self, name, value):
        print(f"Setting value for {name}")
        if name not in self.valid_keys:
            raise ValueError(f'{name} is an invalid attribute!')
        self.__dict__[name] = value 

In [ ]:
john = Entity('John', 49)

In [ ]:
john.names

In [ ]:
vars(john)

In [ ]:
john.names = 10

### Addition and similar operations
- All operations +, -, *, /, //, %, etc map to magic methods in python
- Most of these numeric types also have a corresponding "i" version as well for in place operations
- There are also "r" operations for reflections, i.e. when order is swapped 
- Expected behavior greatly varies based on various object

In [ ]:
class AddExample():

    def __init__(self, value):
        self.value = value
        
    def __add__(self, other):
        """Corresponds to item1 = item1 + other"""
        return AddExample(self.value + other)
    
    def __iadd__(self, other):
        """Corresponds to item1 += other"""
        self.value = self.value + other
        return self
    
    #def __radd__(self, other):
    #    """Corresponds to item1 = other + item1"""
    #    return AddExample(self.value + other)
    
    def __repr__(self):
        """Just as an example..."""
        return str(self.value)

- Add returns a new instance

In [ ]:
sample = AddExample(10)
print(id(sample))

sample = sample + 12
print(id(sample))

print(sample.value)

- Call iadd with +=, for mutable items this will update in place 
- If \__iadd__ is not defined \__add__ will fire

In [ ]:
sample = AddExample(10)
print(id(sample))

sample += 12
print(id(sample))

print(sample.value)

- "reflection" add errors without radd defined

In [ ]:
sample = AddExample(10)
new = 12 + sample
new

##### Continuing our example...

In [ ]:
import numpy as np

class Entity():
    
    def __init__(self, name, risk_score):
        self.name = name 
        self.risk_score = risk_score
        
    def __iadd__(self, other):
        self.risk_score = np.clip(self.risk_score + other, 0, 99)
        return self


In [ ]:
sample = Entity('John', 10)
sample += 90
print(sample.risk_score)

### Hash
- Enable custom objects to join sets or be keys to dictionaries
- Extreme care needs to be exercised here depending on how your code is setup and what is being used for the hash... I.e. potentially easy for values to change and the hash no longer being valid / able to access stored data
- Suggested to implement \__eq__() method as well if using \__hash__()
- Likely you won't want to do this in your own objects...

In [ ]:
class Entity():
    
    def __init__(self, name, risk_score):
        self.name = name 
        self.risk_score = risk_score
    
    def __repr__(self):
        return self.name
        
    def __hash__(self):
        """Return value for hash checks"""
        return hash(self.name)
    
    def __eq__(self, other):
        if type(other) is type(self):
            return self.risk_score == other.risk_score
        else:
            return False

In [ ]:
john = Entity("Name", 49)
sally = Entity("Name", 9)

In [ ]:
dictionary = {}
dictionary[john] = "under review"
dictionary[sally] = "new"
dictionary

In [ ]:
dictionary[sally]

- Change the item being hashed and chaos ensues...

In [ ]:
sally.name = "Sally"

In [ ]:
dictionary[sally]

### Slots
- Add slots for memory performance... treats class like tuple instead of dictionary.. 
- Extra benefit that new instance attributes can not be added
- If you have millions of instances in memory this can be a very efficient option
- Although 3.6+ has lots of dictionary improvements for classes
- Doesn't play totally nice with multiple inheritance 

In [ ]:
class Entity():
    
    __slots__ = ("name", "risk_score")
                  
    def __init__(self, name, risk_score):
        self.name = name 
        self.risk_score = risk_score
        
    def __repr__(self):
        return self.name


In [ ]:
sally = Entity('Sally', 9)

- Can't set new names since the underlying data structure is a tuple

In [ ]:
sally.names = "Sal"

### Lots of other available magic methods...!
- Again, read the docs! 
https://docs.python.org/3/reference/datamodel.html
- Check out https://docs.python.org/3/library/collections.abc.html#collections-abstract-base-classes

# Using Named Tuple for class parent
- Automatically implements several magic methods for us... 

In [ ]:
import typing
class Entity(typing.NamedTuple):
    name: str
    risk_score: int

In [ ]:
john = Entity("John", 9)
johnny = Entity("John", 9)

In [ ]:
if john == johnny:
    print("Built in check against attributes")

In [ ]:
john

# Classmethod

- Way to create a class off of arguments other than what's in the init

In [ ]:
class Entity():
    
    def __init__(self, name, risk_score=0):
        self.name = name 
        self.risk_score = risk_score

    @classmethod
    def from_directory(cls, value):
        """Alternate constructor for Entity class"""
        print(cls.__name__)
        location, title, name = value.split("/")
        entity = cls(name)
        entity.location = location
        return entity

In [ ]:
john = Entity('John', 9)
vars(john)

In [ ]:
sally = Entity.from_directory("AZ/HR Manager/Sally")
vars(sally)

# Property decorator 
- Property decorators are a great tool for lazy evaluation, adding extra logic, and removing fields from instance dictionary 
- Added benefit that you can override names in place so no code changes are required elsewhere
- () is no longer needed on property variables as well, which can have its benefits
- Get, set, and delete can all be defined for a namespace

In [ ]:
import random
def fake_datebase(entity, query):
    return f"{random.randint(0,100)} Hours Ago"

class Entity():
    
    def __init__(self, name, risk_score):
        self.name = name 
        self.risk_score = risk_score
        
    def __repr__(self):
        return f"<Entity Object: {self.name}>"
    
    @property
    def latest_event(self):
        """Lazy database query example with only get set"""
        return fake_datebase(self.name, query="latest event")

In [ ]:
john = Entity('John', 0)

In [ ]:
vars(john)

In [ ]:
john.latest_event

In [ ]:
vars(Entity)

In [ ]:
john.latest_event = "0 Hours Ago"

### Using property variables to check input
- Can even be implemented after the fact using the same namespace    
- Say for instance we want to check values on a predefined attribute...

In [ ]:
class Entity():
    
    def __init__(self, name, risk_score):
        self.name = name 
        self.risk_score = risk_score
        
    def __repr__(self):
        return f"<Entity Object: {self.name}>"
        
    @property
    def risk_score(self):
        """Add property decorator to enable property setters below"""
        print("We can add logic here too if desired...")
        return self._risk_score # _whatever is an arbitrary variable name, just keep this and below the same 
    
    @risk_score.setter
    def risk_score(self, value):
        """"Checks input values off below logic every time variable is set"""
        print("Inside the set")
        if isinstance(value, int) and 0 <= value <= 99:
            self._risk_score = value 
        else:
            raise ValueError("Expecting integer between 0 and 99 inclusive")
                 

In [ ]:
john = Entity('John', 0)
vars(john)

In [ ]:
vars(Entity)

In [ ]:
john.risk_score

In [ ]:
john.risk_score = 10

In [ ]:
vars(john)

- Still can override the "hidden" value if you know the name

In [ ]:
john._risk_score = "21" # john.__dict__["_risk_score"] = "21"
vars(john)

In [ ]:
john.risk_score

# Descriptors
- Descriptors provide the underlying magic for most of Python’s class features, including @classmethod, @staticmethod, @property, and even the \__slots__ specification    
- Descriptors allow for extra logic similar to property setters with less (and reusable) code    
- Using class variables of another class type we create a means to proxy alter the underlying dictionary of instances of the original class 

In [ ]:
import math

class verify_int(object):

    def __init__(self, name, mins=-math.inf, maxs=math.inf):
        self.name = name
        self.mins = mins
        self.maxs = maxs
        
    def __repr__(self):
        return f"<Verify Int Object: {self.name}>"

    def __get__(self, instance, cls):
        """Called everytime we call for the value of an instance"""
        if instance is None:
            print(f"inside verify_int __get__ called from {cls.__name__}")
            return self
        else:
            print(f"inside verify_int __get__ called from {instance}")
            return instance.__dict__[self.name]

    #def __set__(self, instance, value):
    #    """Called everytime instance is assigned a value"""
    #    print(f"inside __set__ called from {instance}, {value}")
    #    if isinstance(value, int) and self.mins <= value <= self.maxs:
    #        instance.__dict__[self.name] = value
    #    else:
    #        raise ValueError(f"Expecting Integer between {self.mins} & {self.maxs}")

- Comment out the \__set__ method above and note the difference in how risk_score is now an attribute vs the class instance of verify_int getting called 

In [ ]:
class Entity():
    
    risk_score = verify_int("example", 0, 99)
    
    def __init__(self, name, risk_score_in):
        self.name = name 
        self.risk_score = risk_score_in
        
    def __repr__(self):
        return f"<Entity Object: {self.name}>"
        

In [ ]:
john = Entity('John', 11)

In [ ]:
john.__dict__

In [ ]:
vars(john)

In [ ]:
Entity.risk_score

In [ ]:
vars(Entity)

- Calling "gets" on instance and class descriptor

In [ ]:
Entity.risk_score

In [ ]:
john.risk_score

- Calling "set" on instance

In [ ]:
john.risk_score = 110

- Again we can still get around checks here after uncommenting above descriptor logic... 
- This is done by using the same mechanism to modify the instance dictionary

In [ ]:
john.__dict__["example"] = "12"
vars(john)

In [ ]:
john.risk_score

- We can also use some builtins to get, set, and check attributes 

In [ ]:
Entity.__dict__

In [ ]:
# Same as hasattr(verify_int, "__set__")
hasattr(Entity.__dict__["risk_score"].__class__, "__set__")

In [ ]:
Entity.__dict__["risk_score"].__class__.__set__

In [ ]:
getattr(john, "risk_score")

In [ ]:
setattr(john, "risk_score", 42)
setattr(john, "name", "Johnny")

In [ ]:
vars(john)

# Property setters under the covers... quick glance
- Property setters are descriptors with some syntactical sugar...
- Basically we use the decorators to pass functions into \__set__ and \__get__ for us

In [ ]:
import random
def fake_datebase(entity, query):
    return f"{random.randint(0,100)} Hours Ago"

class Entity():
    
    def __init__(self, name, risk_score):
        self.name = name 
        self.risk_score = risk_score
        
    def __repr__(self):
        return f"<Entity Object: {self.name}>"
    
    @property
    def latest_event(self):
        """Lazy query database example with only get set"""
        return fake_datebase(self.name, query="latest event")
    
    @property
    def risk_score(self):
        """Add property decorator to enable property setters below"""
        print("We can add logic here too if desired...")
        return self._whatever # _whatever is an arbitrary variable name, just keep this and below the same 
    
    @risk_score.setter
    def risk_score(self, value):
        """"Checks input values off below logic every time variable 'mode' is set"""
        if isinstance(value, int) and 0 <= value <= 99:
            self._whatever = value 
        else:
            raise ValueError("Expecting integer between 0 and 99 inclusive")
                 

In [ ]:
john = Entity('John', 12)
vars(john)

In [ ]:
vars(Entity) # same as project1.__class__.__dict__

- We can check to see if set get or delete are defined in descriptor format
- The real check for property decorators though here is if fset is defined...
- There are fset, fget, and dset (delete) as possible options

In [ ]:
hasattr(Entity.__dict__['latest_event'], '__set__')

In [ ]:
# fset is used to handle actual updating of private variable in question 
bool(Entity.latest_event.fset)

In [ ]:
Entity.latest_event.__set__

- latest_event only has get defined, risk_score has both get and set...

In [ ]:
hasattr(Entity.__dict__['risk_score'], '__set__')

In [ ]:
bool(Entity.risk_score.fset)

In [ ]:
Entity.risk_score.fset